In [1]:
import os
import sys
import subprocess
import datetime as dt
import itertools
import numpy as np
import xarray as xr
from dask.distributed import Client, LocalCluster
import dask.array as da
import matplotlib.pyplot as plt
from MHW_daily_calcs import *

In [2]:
climyrs=[1993,2023]
windowhalfwid=10
smoothmethod='tri'
jj=60
qtile=.9
detr=True
smoothedClim=True
smoothedTrend=True
meth=smoothmethod
win=windowhalfwid
delt=30
qtvar='qt1'
ilead=40

In [3]:
    lmax=215
    def getind(i0):
        if i0>=1 and i0<=10:
            return [i0-1,i0,i0+1]
        elif i0==0:
            return [11,0,1]
        elif i0==11:
            return [10,11,0]
    def leadbounds(l0,lmax,delt):
        return max(0,l0-delt), min(lmax,l0+delt+1)
    if detr:
        flist=[fnameCanESMAnomDetrByLead(workdir, climyrs[0],climyrs[-1],il,jj,smoothClim=smoothedClim,smoothTrend=smoothedTrend,meth=smoothmethod,win=win) \
                for il in range(*leadbounds(ilead,215,delt))]
    else:
        flist=[fnameCanESMAnomByLeadNoDetr(workdir, climyrs[0], climyrs[-1], il, jj,smoothClim=smoothedClim,meth=smoothmethod,win=window) \
                for il in range(*leadbounds(ilead,215,delt))]
    print(flist)
    fqout=fnameCanESMAnomQtile(workdir, climyrs[0], climyrs[-1], ilead, jj, qtile, detr,
                               smoothClim=smoothedClim,smoothTrend=smoothedTrend,meth=smoothmethod,win=win,delt=delt)
    ff=xr.open_mfdataset(flist,combine='nested',concat_dim=['leadtime'],parallel=True,decode_times=False)
    fc=ff.sst_an.coarsen(reftime=12,boundary='pad').construct(reftime=('year','month'))
    sh=fc.shape

['/space/hall5/sitestore/eccc/crd/ccrn/users/reo000/work/MHW_daily/byLeadDetr/anomDetrByLead_ClimStri10_TrStri10_cwao_CanESM5.1p1bc-v20240611_hindcast_C1993_2023_L010_j060_ocean_1d_surface_tso.nc', '/space/hall5/sitestore/eccc/crd/ccrn/users/reo000/work/MHW_daily/byLeadDetr/anomDetrByLead_ClimStri10_TrStri10_cwao_CanESM5.1p1bc-v20240611_hindcast_C1993_2023_L011_j060_ocean_1d_surface_tso.nc', '/space/hall5/sitestore/eccc/crd/ccrn/users/reo000/work/MHW_daily/byLeadDetr/anomDetrByLead_ClimStri10_TrStri10_cwao_CanESM5.1p1bc-v20240611_hindcast_C1993_2023_L012_j060_ocean_1d_surface_tso.nc', '/space/hall5/sitestore/eccc/crd/ccrn/users/reo000/work/MHW_daily/byLeadDetr/anomDetrByLead_ClimStri10_TrStri10_cwao_CanESM5.1p1bc-v20240611_hindcast_C1993_2023_L013_j060_ocean_1d_surface_tso.nc', '/space/hall5/sitestore/eccc/crd/ccrn/users/reo000/work/MHW_daily/byLeadDetr/anomDetrByLead_ClimStri10_TrStri10_cwao_CanESM5.1p1bc-v20240611_hindcast_C1993_2023_L014_j060_ocean_1d_surface_tso.nc', '/space/hall5/

In [4]:
fc=fc.chunk({'lat':10,'lon':10})
fc

The history saving thread hit an unexpected error (OperationalError('disk I/O error')).History will not be written to the database.


<xarray.DataArray 'sst_an' (leadtime: 61, year: 32, month: 12, r: 20, lat: 60,
                            lon: 360)> Size: 81GB
dask.array<rechunk-merge, shape=(61, 32, 12, 20, 60, 360), dtype=float64, chunksize=(1, 31, 12, 20, 10, 10), chunktype=numpy.ndarray>
Coordinates:
  * lat       (lat) float64 480B -29.5 -28.5 -27.5 -26.5 ... 26.5 27.5 28.5 29.5
  * leadtime  (leadtime) float64 488B 255.0 279.0 303.0 ... 1.671e+03 1.695e+03
  * lon       (lon) float64 3kB 0.5 1.5 2.5 3.5 4.5 ... 356.5 357.5 358.5 359.5
    reftime   (year, month) float64 3kB dask.array<chunksize=(32, 12), meta=np.ndarray>
  * r         (r) int64 160B 0 1 2 3 4 5 6 7 8 9 10 11 12 13 14 15 16 17 18 19
Dimensions without coordinates: year, month

In [5]:
    ql1=np.nan*da.ones((12,sh[-2],sh[-1]))
    ql2=np.nan*da.ones((12,sh[-2],sh[-1]))
    ql1

dask.array<mul, shape=(12, 60, 360), dtype=float64, chunksize=(12, 60, 360), chunktype=numpy.ndarray>

In [6]:
ii=0
pool1=fc.isel(month=ii).data.reshape((sh[0]*sh[1]*sh[3],sh[4],sh[5])).rechunk((-1,60,30))
pool1

dask.array<rechunk-merge, shape=(39040, 60, 360), dtype=float64, chunksize=(39040, 60, 30), chunktype=numpy.ndarray>

In [7]:
ql1[ii,...]=da.apply_along_axis(np.quantile,0,pool1,qtile).compute()

In [8]:
ql1[0,30,200].compute()

np.float64(1.6830794949765082)

In [9]:
temp=pool1[:,30,200].compute()

In [10]:
np.shape(temp)

(39040,)

In [11]:
np.quantile(temp,qtile)

np.float64(1.6830794949765082)

In [12]:

    for ii in range(0,12):
        pool1=fc.isel(month=ii).data.reshape((sh[0]*sh[1]*sh[3],sh[4],sh[5]))
        ql1[ii,...]=pool1.quantile(qtile,axis=0)
        pool2=fc.sel(month=getind(ii)).data.reshape((sh[0]*sh[1]*3*sh[3],sh[4],sh[5]))
        ql2[ii,...]=pool2.quantile(qtile,axis=0)
    print(fqout)

AttributeError: 'Array' object has no attribute 'quantile'

In [16]:
if detr:
    flist=[fnameOISSTAnomDetr([ylimlistobs[0][0],ylimlistobs[-1][-1]],climyrs, jj, smoothClim, meth, win),]
else:
    flist=[fnameOISSTAnom(yrlims, climyrs, jj, smoothClim, meth, win) for yrlims in ylimlistobs]
print(flist)
fanom=xr.open_mfdataset(flist,parallel=True,decode_times=False)
fqtile= fnameOISSTQTile(climyrs, jj, qtile, smoothClim, meth, win,detr,delt)
fMHW = fnameOISSTMHW(climyrs, jj, qtile, smoothClim, meth, win,detr,delt,qtvar)

# real data has leap years, so coarsen won't work; need to account for 366 day years as well
tdt=np.array([dt.datetime(1978,1,1,12)+dt.timedelta(days=float(el)) for el in fanom.time.values])
yd=[yd365(el) for el in tdt]
fq=xr.open_dataset(fqtile,decode_times=False)

['/space/hall5/sitestore/eccc/crd/ccrn/users/reo000/work/MHW_daily/OISST/oisst_anom_detr_ClimStri10_C1993_2023-avhrr-v02r01.regridded1x1g2.daily.1991_2024_j60.nc']


In [27]:
qt2=fq[qtvar].sel(yd=yd)

In [32]:
MHW=np.ma.masked_where(np.logical_or(np.isnan(fanom['sst_an'].values),np.isnan(fanom['sst_an'].values)),
                       np.where(fanom['sst_an'].values>qt2.values,1,0))

In [31]:
MHW.shape

(12235, 60, 360)

In [33]:
fanom

<xarray.Dataset> Size: 2GB
Dimensions:  (time: 12235, lat: 60, lon: 360)
Coordinates:
  * time     (time) float32 49kB 4.748e+03 4.749e+03 ... 1.698e+04 1.698e+04
  * lat      (lat) float32 240B -29.5 -28.5 -27.5 -26.5 ... 26.5 27.5 28.5 29.5
  * lon      (lon) float32 1kB 0.5 1.5 2.5 3.5 4.5 ... 356.5 357.5 358.5 359.5
Data variables:
    sst_an   (time, lat, lon) float64 2GB dask.array<chunksize=(12235, 60, 360), meta=np.ndarray>

In [17]:
test=fq[qtvar].sel(yd=yd)

In [20]:
fanom['sst_an']

<xarray.DataArray 'sst_an' (time: 12235, lat: 60, lon: 360)> Size: 2GB
dask.array<open_dataset-sst_an, shape=(12235, 60, 360), dtype=float64, chunksize=(12235, 60, 360), chunktype=numpy.ndarray>
Coordinates:
  * time     (time) float32 49kB 4.748e+03 4.749e+03 ... 1.698e+04 1.698e+04
  * lat      (lat) float32 240B -29.5 -28.5 -27.5 -26.5 ... 26.5 27.5 28.5 29.5
  * lon      (lon) float32 1kB 0.5 1.5 2.5 3.5 4.5 ... 356.5 357.5 358.5 359.5

In [18]:
test

<xarray.DataArray 'qt1' (yd: 12235, lat: 60, lon: 360)> Size: 2GB
[264276000 values with dtype=float64]
Coordinates:
  * lat      (lat) float32 240B -29.5 -28.5 -27.5 -26.5 ... 26.5 27.5 28.5 29.5
  * lon      (lon) float32 1kB 0.5 1.5 2.5 3.5 4.5 ... 356.5 357.5 358.5 359.5
  * yd       (yd) int64 98kB 1 2 3 4 5 6 7 8 ... 175 176 177 178 179 180 181 182
Attributes:
    long_name:  90.0th percentile value

In [ ]:
    qt2=np.zeros(np.shape(ff['sst_an'].values))
    for il in yd:
        qt2[il,...]=fq[qtvar].sel(yd=il)